# Baseline system of DBO detection model 

The Jupyter notebook `dbo_baseline.ipynb` contains the code for creating the baseline system for fine-grained detection of various types of attacks on the **free democratic basic order (DBO)** (**subtask 2** of the Shared Task on Harmful Content Detection). An Support Vector Machine (SVM) with a linear kernel using TF-IDF features was chosen as the classifier. The notebook includes the training of the system as well as the prediction on the test data and the evaluation. 

The programme was tested using Python version 3.12.9. Executing the following two lines of code will install all necessary packages. 

In [1]:
%%writefile requirements.txt

pandas==2.2.3
spacy==3.8.2
scikit-learn==1.6.1
textblob==0.15.3
textblob-de==0.4.3
sentence-transformers==4.1.0
nltk==3.9.1
numpy==2.0.1

Overwriting requirements.txt


In [ ]:
%pip install -r requirements.txt 

## 1. Data preprocessing

First, the training data was read in. 

In [3]:
import pandas as pd

filename = 'dbo_train.csv' # Path needs to be adjusted  

# Reading in training data
train_dbo = pd.read_csv(filename, sep=';')
train_dbo.drop('id', axis=1, inplace=True)
train_dbo.head()

,description,DBO
0,"Der Riese ist geweckt,mit oder ohne Verräter u...",nothing
1,Gut Ding will Weile haben... (y),nothing
2,Sollen sie doch nach Saudi Arabien,nothing
3,Volle Zustimmung.??,nothing
4,Mal sehen wann wir an der Erderwärmung schuld ...,nothing


The class distribution of the training data was analysed. 

In [4]:
# Absolute number of instances in each class 
class_counts_dbo = train_dbo["DBO"].value_counts()

# Relative number of instances in each class 
class_percent_dbo = train_dbo['DBO'].value_counts(normalize=True) * 100

# Summarise into a dataframe
class_table_dbo = pd.DataFrame({
    'Frequency': class_counts_dbo,
    'Percentage': class_percent_dbo.round(2)
})

print(class_table_dbo)

            Frequency  Percentage
DBO                              
nothing          6277       84.21
criticism         804       10.79
agitation         313        4.20
subversive         60        0.80


The training data was then pre-processed. First, basic cleaning steps (removing URLs, mentions, hashtags, numbers and redundant whitespace) were carried out.

In [5]:
import re


def text_clean(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', ' NUM ', text)  # Replace numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text


# Application of pre-processing steps to the training data
train_dbo['description'] = train_dbo['description'].apply(text_clean)
train_dbo.head()

,description,DBO
0,"der riese ist geweckt,mit oder ohne verräter u...",nothing
1,gut ding will weile haben... (y),nothing
2,sollen sie doch nach saudi arabien,nothing
3,volle zustimmung.??,nothing
4,mal sehen wann wir an der erderwärmung schuld ...,nothing


Furthermore, the class labels (*agitation*, *criticism*, *nothing*, *subversive*) were mapped to numerical values (0, 1, 2, 3). 

In [6]:
# Encode labels
train_dbo['DBO_encoded']= train_dbo['DBO'].apply(lambda x: ['agitation', 'criticism', 'nothing', 'subversive'].index(x))

## 2. TF-IDF vectorization

TF-IDF weighted bag-of-phrases representation was chosen as the feature, with tweets tokenised into unigrams and bigrams. The vocabulary comprised the 5,000 most common terms in the training data set. 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train = vectorizer.fit_transform(train_dbo['description'])
y_train = train_dbo['DBO_encoded']
print(X_train.shape, y_train.shape)

(7454, 5000) (7454,)


## 3. Model training 

To counteract the strong class imbalance in the training data set, a cost-sensitive SVM was used. Misclassifications for the underrepresented classes (especially subversion and agitation) were penalised more heavily than for the other two classes to prevent the Support Vector Machine (SVM) from assigning the tweets only to the overrepresented classes. The class weights were initially calculated based on the proportion of instances of this class in the training data set. 

In [8]:
import numpy as np
from sklearn.utils import compute_class_weight

# Calculate the class weights for each class
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
class_weights_dict

{0: np.float64(5.953674121405751),
 1: np.float64(2.317786069651741),
 2: np.float64(0.2968774892464553),
 3: np.float64(31.058333333333334)}

Subsequently, parameter C, which determines the data points of which class the classifier must pay particular attention to for correct classification, was set for each class depending on the calculated class weights. An SVM with linear kernel was trained using the TF-IDF representation of the tweets. 

In [ ]:
from sklearn.svm import SVC

# Training svm 
svm_model = SVC(kernel='linear', class_weight=class_weights_dict)
svm_model.fit(X_train, y_train)

## 5. Prediction on test data

Predictions were then made on the test data using the trained SVM model. For this purpose, the test data was preprocessed in the same way as the training data. 

In [10]:
# Importing the test data
filename = "dbo_test.csv" # Path needs to be adjusted 
test_dbo = pd.read_csv(filename, sep=';')

In [11]:
# Cleaning up the test data
test_dbo['description'] = test_dbo['description'].apply(text_clean)

The test data was then also TF-IDF vectorised (using the vocabulary from the training data). 

In [12]:
# TF-IDF vectorisation of test data
test_dbo_idf_vec = vectorizer.transform(test_dbo['description'])

The extracted features of the test data are passed to the gradient boosting model for prediction. 

In [13]:
# Prediction 
y_pred = svm_model.predict(test_dbo_idf_vec)

## 6. Model evaluation

The predictions based on the test data were compared with the gold standard and some basic evaluation metrics were calculated. The results achieved serve as a guide and baseline for the competition participants. 

In [14]:
# Importing the gold standard 
filename = "dbo_gold.csv" # Path needs to be adjusted 
gold_dbo = pd.read_csv(filename, sep=';')

In [15]:
# Encode the labels 
gold_dbo['DBO_encoded']= gold_dbo['DBO'].apply(lambda x: ['agitation', 'criticism', 'nothing', 'subversive'].index(x))

In [16]:
# Extracting the actual label of the test data
y_true = gold_dbo['DBO_encoded']

The macro metric F1 serves as the main evaluation metric used to calculate the ranking on the leaderboard for the competition. In addition, other evaluation metrics such as precision and recall are calculated for the individual classes, as well as the macro and weighted average. 

In [17]:
from sklearn.metrics import classification_report

y_pred = svm_model.predict(test_dbo_idf_vec)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.46      0.36       134
           1       0.36      0.65      0.47       345
           2       0.94      0.82      0.88      2690
           3       0.60      0.12      0.20        25

    accuracy                           0.78      3194
   macro avg       0.55      0.51      0.47      3194
weighted avg       0.85      0.78      0.80      3194

